# Load PDF and Split into Chunks

In [1]:
# !pip uninstall -y protobuf
# !pip install protobuf==6.31.1 --force-reinstall --no-cache-dir

# import google.protobuf
# print(google.protobuf.__version__)

!pip install langchain_community pymupdf
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [2]:
pdf_path = "/content/APJAbdulKalam.pdf"  # 🔁 Replace this with your PDF path

loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

In [3]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(docs)
documents = [doc.page_content for doc in split_docs]
metadatas = [{"source": doc.metadata.get("page", "unknown")} for doc in split_docs]


In [35]:
len(documents)
split_docs

[Document(metadata={'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2015-06-24T14:50:21+05:30', 'source': '/content/APJAbdulKalam.pdf', 'file_path': '/content/APJAbdulKalam.pdf', 'total_pages': 5, 'format': 'PDF 1.5', 'title': 'Profile of Dr', 'author': 'HQ WEST COMD HUT', 'subject': '', 'keywords': '', 'moddate': '2015-06-24T14:50:21+05:30', 'trapped': '', 'modDate': "D:20150624145021+05'30'", 'creationDate': "D:20150624145021+05'30'", 'page': 0}, page_content='Dr. APJ Abdul Kalam \nwww.abdulkalam.com \nPage 1 of 5 \nProfile of Dr. APJ Abdul Kalam \n \n Born on 15th October 1931 at \nRameswaram in Tamil Nadu, Dr. Avul \nPakir Jainulabdeen Abdul Kalam, \ngraduated \nin \nScience \nfrom \nSt. \nJoseph’s College, Trichy in 1954 and \nspecialized \nin \nAeronautical \nEngineering from Madras Institute of \nTechnology (MIT) in 1957. Dr. Kalam \nis a pioneer in fibre glass technology and led a young team to \ninitiate this effort in ISRO from design, 

# Embedding Model - Jina AI

In [5]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

In [6]:
embedding_model_name = "jinaai/jina-embeddings-v2-base-code"
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name, trust_remote_code=True)
embedding_model = AutoModel.from_pretrained(embedding_model_name, trust_remote_code=True)

def get_embedding(texts):
    inputs = embedding_tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = embedding_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/root/.cache/huggingface/modules/transformers_modules/jinaai/jina-bert-v2-qk-post-norm/3baf9e3ac750e76e8edd3019170176884695fb94/configuration_bert.py:29: UserWarning: optimum is not installed. To use OnnxConfig and BertOnnxConfig, make sure that `optimum` package is installed
  warnings.warn("optimum is not installed. To use OnnxConfig and BertOnnxConfig, make sure that `optimum` package is installed")


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [7]:
embeddings = get_embedding(documents)
embeddings

array([[ 0.36558867,  0.97121215, -0.70632607, ..., -0.29313993,
        -0.3028977 ,  0.19185384],
       [ 0.64828473,  0.24969731, -0.34193248, ...,  0.25982213,
        -0.14664166,  0.1493164 ],
       [ 1.419285  ,  0.05153485, -0.32390973, ...,  0.22935256,
        -0.4485155 ,  0.19096914],
       ...,
       [ 0.04847092,  0.6058839 , -0.9268254 , ..., -0.58594304,
        -0.1099766 ,  0.49540502],
       [ 0.4140246 ,  1.23072   , -0.41796798, ..., -0.43438384,
        -0.22568695,  0.91806453],
       [ 0.7714628 ,  1.0265807 , -0.30603516, ..., -0.31390467,
         0.02288589,  1.0789241 ]], dtype=float32)

# Chroma Database

In [8]:
# # Step 2: Restart the runtime (IMPORTANT)
# import os
# os.kill(os.getpid(), 9)  # force restarts the Colab runtime

!pip install chromadb
from chromadb import Client
from chromadb.config import Settings

import google.protobuf
print(google.protobuf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.73.0 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.
grpcio-health-checking 1.73.0 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.


6.31.1


/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.5 is exactly one major version older than the runtime version 6.31.1 at google/protobuf/duration.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [9]:
chroma_client = Client(Settings(anonymized_telemetry=False, persist_directory="./chroma_jina_db"))
collections = chroma_client.list_collections()

for collection in collections:
    print(f"Deleting collection: {collection.name}")
    chroma_client.delete_collection(name=collection.name)

chroma_collection = chroma_client.create_collection(name="Chroma_Collection")

chroma_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=[f"doc{i}" for i in range(len(documents))],
    embeddings=embeddings.tolist()
)

# Pinecone Database

In [10]:
!pip install pinecone-client==4.0.0

In [11]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_3p3bSz_5MEkGJeGRtMMiq57Zt9cXtcVMaN3UvmyokxYvYurnV4SdP1dg4ZnSoQGcyRMBXL")
index_name = "test2"

index = pc.Index(index_name)

pinecone_data = [
    {
        "id": f"doc{i}",
        "values": embeddings[i].tolist(),
        "metadata": {
            "text": documents[i],
            "source": metadatas[i]["source"]
        }
    }
    for i in range(len(documents))
]

index.upsert(vectors=pinecone_data)

{'upserted_count': 18}

# Weviate Database

In [12]:
!pip install -U weaviate-client

  Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl (321 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.34.1 requires protobuf<6.0,>=5.0, but you have protobuf 6.31.1 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.2

In [13]:
WEAVIATE_URL="8nzs5auetcay3wetgzuxjw.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="T2t0UCtJSlJPemdjcUgyYl9BSDVXNXZIeE9UVURFOHlLNUh6ZjlPWU5OdG9zQnBLVjBQSVJsbmdLMDFjPV92MjAw"

In [14]:
import weaviate
from weaviate.classes.init import Auth
import os
import uuid

# Best practice: store your credentials in environment variables
weaviate_url = WEAVIATE_URL
weaviate_api_key = WEAVIATE_API_KEY

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())  # Should print: `True`

True


In [15]:
COLLECTION_NAME="test1"
weviate_collection = client.collections.get(COLLECTION_NAME)

In [16]:
with weviate_collection.batch.fixed_size(batch_size=500) as batch:
    for doc, meta, embedding in zip(documents, metadatas, embeddings):
        batch.add_object(
            properties={
                "text": doc,
                "source": str(meta["source"])
            },
            vector=embedding.tolist(),
            uuid=str(uuid.uuid4())
        )
        if batch.number_errors > 10:
            print("❌ Too many errors in batch, stopping upload.")
            break

# Optional: Check failed uploads
if weviate_collection.batch.failed_objects:
    print("❗ Failed imports:", len(weviate_collection.batch.failed_objects))

# Query

In [25]:
query = "What are the achievements of abdul kalam?"
query_embedding_vector = get_embedding([query])[0].tolist()

# Chroma Results

In [26]:
chroma_results = chroma_collection.query(
    query_embeddings=query_embedding_vector,
    n_results=3
)
chroma_relavant_docs = chroma_results["documents"][0]
chroma_scores = chroma_results["distances"][0]

print("\n🔍 Retrieved Chunks from Chroma:")
for doc, score in zip(chroma_relavant_docs, chroma_scores):
    print(f"- Score: {score:.4f}")
    print(f"  Text: {doc.strip()}")
    print()


🔍 Retrieved Chunks from Chroma:
- Score: 139.6587
  Text: responsible for evolving policies, strategies and missions for 
many development applications. Dr. Kalam was also the 
Chairman, Ex-officio, of the Scientific Advisory Committee to 
the Cabinet (SAC-C) and piloted India Millennium Mission 
2020.

- Score: 177.5458
  Text: initiated the use of carbon-carbon and carbon-polymer 
materials for production of floor reaction orthosis calipers 
which has reduced the weight of the caliper to 1/10th of the 
original weight during 1995 – 1996. Over 50,000 children have 
been fitted with these calipers.  
Dr. Kalam took up academic pursuit as Professor, 
Technology & Societal Transformation at Anna University, 
Chennai from November 2001 and was involved in teaching

- Score: 193.0533
  Text: initiate this effort in ISRO from design, development leading to 
production of composites rocket motor cases. Dr. Kalam made 
significant contribution as Project Director to develop India’s 
first in

# Pinecone Results

In [27]:
pinecone_results = index.query(vector=query_embedding_vector, top_k=3, include_metadata=True)

pinecone_relavant_docs = [(match['metadata']['text'], match['score']) for match in pinecone_results['matches']]

print("\n🔍 Retrieved Chunks from Pinecone:")
for doc, score in pinecone_relavant_docs:
    print(f"- Score: {score:.4f}")
    print(f"  Text: {doc}")
    print()


🔍 Retrieved Chunks from Pinecone:
- Score: 0.6159
  Text: responsible for evolving policies, strategies and missions for 
many development applications. Dr. Kalam was also the 
Chairman, Ex-officio, of the Scientific Advisory Committee to 
the Cabinet (SAC-C) and piloted India Millennium Mission 
2020.

- Score: 0.4935
  Text: initiated the use of carbon-carbon and carbon-polymer 
materials for production of floor reaction orthosis calipers 
which has reduced the weight of the caliper to 1/10th of the 
original weight during 1995 – 1996. Over 50,000 children have 
been fitted with these calipers.  
Dr. Kalam took up academic pursuit as Professor, 
Technology & Societal Transformation at Anna University, 
Chennai from November 2001 and was involved in teaching

- Score: 0.4673
  Text: Assessment Council (TIFAC) and as an eminent scientist, he 
led the country with the help of 500 experts to arrive at 
Technology Vision 2020 giving a road map for transforming 
India from the present dev

# Weviate Results

In [28]:
weviate_results = weviate_collection.query.near_vector(
    near_vector=query_embedding_vector,
    limit=3,
    return_metadata=["distance"]
)

weviate_relavant_docs = [
    (obj.properties["text"], obj.metadata.distance)
    for obj in weviate_results.objects
]

print("\n🔍 Retrieved Chunks from Weaviate:")
for doc, distance in weviate_relavant_docs:
    print(f"- Score: {distance:.4f}")
    print(f"  Text: {doc}")
    print()



🔍 Retrieved Chunks from Weaviate:
- Score: 0.3841
  Text: responsible for evolving policies, strategies and missions for 
many development applications. Dr. Kalam was also the 
Chairman, Ex-officio, of the Scientific Advisory Committee to 
the Cabinet (SAC-C) and piloted India Millennium Mission 
2020.

- Score: 0.3841
  Text: responsible for evolving policies, strategies and missions for 
many development applications. Dr. Kalam was also the 
Chairman, Ex-officio, of the Scientific Advisory Committee to 
the Cabinet (SAC-C) and piloted India Millennium Mission 
2020.

- Score: 0.3841
  Text: responsible for evolving policies, strategies and missions for 
many development applications. Dr. Kalam was also the 
Chairman, Ex-officio, of the Scientific Advisory Committee to 
the Cabinet (SAC-C) and piloted India Millennium Mission 
2020.



| DB       | Min (Best) | Max (Worst) |
| -------- | ---------- | ----------- |
| Chroma   | 0.0        | 150–300+    |
| Pinecone | 0.7–1.0    | 0.0–0.6     |
| Weaviate | 0.0        | 2–10        |

# LLM

In [29]:
from transformers import pipeline

In [31]:
llm_model_name = "google/flan-t5-base"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)

llm_pipe = pipeline(
    "text2text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    max_length=512,
    device=0 if torch.cuda.is_available() else -1
)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [32]:
chroma_context = "\n\n".join([
    f"Chunk {i+1}:\n{text.strip()}"
    for i, text in enumerate(chroma_relavant_docs)
])

print(f"\n🧠 Question: {query}")

final_input = f"""You are a highly knowledgeable assistant.
Your job is to answer the following question **only** based on the provided context.

Summarize the identity, contributions, and role of the person mentioned in the question.

Context:
{chroma_context}

Question: {query}
Answer:"""

chroma_response = llm_pipe(final_input)[0]["generated_text"]
print("\n✅ Chroma Answer:\n", chroma_response)


🧠 Question: What are the achievements of abdul kalam?

✅ Chroma Answer:
 Dr. Kalam was also the Chairman, Ex-officio, of the Scientific Advisory Committee to the Cabinet (SAC-C) and piloted India Millennium Mission 2020.


In [33]:
pinecone_context = "\n\n".join([
    f"{text.strip()}"
    for text, score in pinecone_relavant_docs
])

print(f"\n🧠 Question: {query}")

final_input = f"""
You are an expert assistant.
Answer the following question strictly based on the provided context."

Context:
{pinecone_context}

Question: {query}
Answer:"""

pinecone_response = llm_pipe(final_input)[0]["generated_text"]
print("\n✅ Pinecone Answer:\n", pinecone_response)


🧠 Question: What are the achievements of abdul kalam?

✅ Pinecone Answer:
 he led the country with the help of 500 experts to arrive at Technology Vision 2020 giving a road map for transforming India from the present developing status to a developed nation


In [34]:
weviate_context = "\n\n".join([
    f"{text.strip()}"
    for text, score in weviate_relavant_docs
])

print(f"\n🧠 Question: {query}")

final_input = f"""
You are an expert assistant.
Answer the following question strictly based on the provided context."

Context:
{weviate_context}

Question: {query}
Answer:"""

weviate_response = llm_pipe(final_input)[0]["generated_text"]
print("\n✅ Weviate Answer:\n", weviate_response)


🧠 Question: What are the achievements of abdul kalam?

✅ Weviate Answer:
 Dr. Kalam was also the Chairman, Ex-officio, of the Scientific Advisory Committee to the Cabinet (SAC-C) and piloted India Millennium Mission 2020
